<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/PerformSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

De Serialze the saved model

In [ ]:

#Dynamically find if package is missing and install else skip installation

import json
import sys
import subprocess
from traceback import format_exc
from typing import Text
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)


!pip3 install gensim

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
outputFolder='/content/gdrive/My Drive/DissertationFiles/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
csvWritePath = outputFolder+'pandasDataFrame.csv'
# #Read Dataframe from CSV file
urlDetails = pd.read_csv(csvWritePath)

In [ ]:
urlDetails.head()

,Url,TextInfo,tokenizedWords,tokenizedScentence,POS
0,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung 1980,"['kabinettsprotokolle', 'bundesregierung', '19...",kabinettsprotokolle bundesregierung 1980,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
1,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 7 195...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
2,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 21 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
3,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 25 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
4,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 19 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."


In [ ]:
import gensim

tfidf_corpus = gensim.corpora.MmCorpus(outputFolder+'tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus(outputFolder+'lsi_model_mm')

print(tfidf_corpus)
print(lsi_corpus)

MmCorpus(61 documents, 3081 features, 5182 non-zero entries)
MmCorpus(61 documents, 55 features, 2494 non-zero entries)


In [ ]:
from gensim.similarities import MatrixSimilarity

search_index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

In [ ]:
from operator import itemgetter

def search_similar_text(search_term):

    query_bow = dictionary.doc2bow(nltk.word_tokenize(search_term))
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    search_index.num_best = 5

    search_list = search_index[query_lsi]

    search_list.sort(key=itemgetter(1), reverse=True)
    search_names = []

    for j, searchResult in enumerate(search_list):
      relavenceFactor =round((searchResult[1] * 100),2)
      if relavenceFactor > 2:
          search_names.append (
              {
                  'Relevance': round((searchResult[1] * 100),2),
                  'Url': urlDetails['Url'][searchResult[0]],
                  'Text': urlDetails['TextInfo'][searchResult[0]]
              }

          )
          if j == (search_index.num_best-1):
              break

    return pd.DataFrame(search_names, columns=['Relevance','Url','Text'])

In [ ]:
df = search_similar_text('Bildungsnotstand konfrontiert')
df.head()

NameError: ignored

In [ ]:
from translate import Translator


def translateTextToEnglish(text2Trans):
  translator= Translator(from_lang="de", to_lang="en")
  translation = translator.translate(text2Trans).replace("&#39;","'")
  return translation


def translateTextToGerman(text2Trans):
  translator= Translator(from_lang="en", to_lang="de")
  translation = translator.translate(text2Trans).replace("&#39;","'")
  return translation
    

In [ ]:
urlDetails.tokenizedScentence[10]


In [ ]:
translateText(urlDetails.tokenizedScentence[10])


In [ ]:
translateTextToEnglish('Bildungsnotstand konfrontiert')

In [ ]:
for scentence in nltk.sent_tokenize(df.Text[0]):
  print(translateTextToEnglish(scentence))
